# 🐻 Bear Detector

**Binary image classification** (MobileNetV2) + **object detection in video** (YOLOv8)  
for wildlife camera-trap footage.

> Run in **Google Colab** (GPU recommended) or a local environment.  
> All configurable parameters live in the first code cell.


## 0 · Setup

In [ ]:
import os, sys
from pathlib import Path

# ── Colab: clone repo and cd into it ─────────────────────────────────────────
IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    !git clone --quiet https://github.com/danort92/Bear-Detector.git
    os.chdir("Bear-Detector")

# ── Local: make sure we are at the repo root ──────────────────────────────────
REPO_ROOT = Path.cwd()
# Add src/ to the Python path so bear_detector is importable
if str(REPO_ROOT / "src") not in sys.path:
    sys.path.insert(0, str(REPO_ROOT / "src"))

print("Working directory:", REPO_ROOT)


## 1 · Install Dependencies

In [ ]:
# Install once; comment out if dependencies are already installed.
%pip install -q -r requirements.txt


## 2 · Configuration

All hyperparameters and paths are centralised here.

In [ ]:
from bear_detector.config import ClassificationConfig, DetectionConfig

clf_cfg = ClassificationConfig(
    bear_dir="ct/bear_ct",
    other_dir="ct/other_ct",
    epochs=20,
    learning_rate=1e-4,
    threshold=0.3,        # P(bear) >= threshold  →  "Bear"
)

det_cfg = DetectionConfig(
    epochs=50,
    image_size=640,
    batch_size=16,
)

print("Classification config:", clf_cfg)
print("Detection config:", det_cfg)


## 3 · Imports

In [ ]:
import os, shutil, zipfile
from datetime import datetime
from pathlib import Path

import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import yaml
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.utils import class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import Recall

from bear_detector.utils import load_and_preprocess, batch_from_path, collect_images_from_dir, extract_zip
from bear_detector.classification import prepare_split_dirs, build_model, train_classification_model, predict_image
from bear_detector.detection import setup_local_dataset, setup_roboflow_dataset, train_detection_model, process_video

# Optional: Colab-specific display helpers
try:
    from google.colab.patches import cv2_imshow
    from google.colab import files as colab_files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False
    print("Not running in Colab — file upload/download cells will be skipped.")

print("TensorFlow:", tf.__version__)


## 4 · Classification Dataset

### 4.1 Train/Val Split

In [ ]:
n_train_bear, n_val_bear, n_train_other, n_val_other = prepare_split_dirs(
    bear_dir=clf_cfg.bear_dir,
    other_dir=clf_cfg.other_dir,
    train_dir=clf_cfg.train_dir,
    val_dir=clf_cfg.val_dir,
    val_split=clf_cfg.val_split,
    random_seed=clf_cfg.random_seed,
)

print(f"Training   — bear: {n_train_bear:4d} | other: {n_train_other:4d}")
print(f"Validation — bear: {n_val_bear:4d} | other: {n_val_other:4d}")


### 4.2 Data Generators

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=clf_cfg.rotation_range,
    width_shift_range=clf_cfg.width_shift_range,
    height_shift_range=clf_cfg.height_shift_range,
    shear_range=clf_cfg.shear_range,
    zoom_range=clf_cfg.zoom_range,
    horizontal_flip=clf_cfg.horizontal_flip,
)
val_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    clf_cfg.train_dir,
    target_size=clf_cfg.image_size,
    batch_size=clf_cfg.batch_size,
    class_mode="binary",
)
val_generator = val_datagen.flow_from_directory(
    clf_cfg.val_dir,
    target_size=clf_cfg.image_size,
    batch_size=clf_cfg.batch_size,
    class_mode="binary",
    shuffle=False,
)

print("Class indices:", train_generator.class_indices)
# Expected: {'bear': 0, 'other': 1}
# Model output = P(other); P(bear) = 1 - output


### 4.3 Visualise Dataset Samples

In [ ]:
def plot_generator_samples(
    generator: ImageDataGenerator,
    num_images: int = 20,
    title: str = "",
) -> None:
    """Plot a random sample of images from a Keras image generator.

    Args:
        generator: A ``DirectoryIterator`` (from ``flow_from_directory``).
        num_images: How many images to display.
        title: Optional figure title.
    """
    generator.reset()
    class_labels = {v: k for k, v in generator.class_indices.items()}

    all_images: list[np.ndarray] = []
    all_labels: list[float] = []
    for _ in range(len(generator)):
        imgs, lbls = next(generator)
        all_images.append(imgs)
        all_labels.append(lbls)

    all_images_arr = np.concatenate(all_images, axis=0)
    all_labels_arr = np.concatenate(all_labels, axis=0)

    num_images = min(num_images, len(all_images_arr))
    indices = np.random.choice(len(all_images_arr), num_images, replace=False)

    cols = 5
    rows = (num_images + cols - 1) // cols
    plt.figure(figsize=(15, rows * 3))
    if title:
        plt.suptitle(title, fontsize=14, y=1.02)

    for plot_idx, img_idx in enumerate(indices):
        label_name = class_labels[int(all_labels_arr[img_idx])]
        plt.subplot(rows, cols, plot_idx + 1)
        plt.imshow(all_images_arr[img_idx])
        plt.title(label_name, fontsize=9)
        plt.axis("off")

    plt.tight_layout()
    plt.show()


plot_generator_samples(train_generator, num_images=20, title="Training samples")
plot_generator_samples(val_generator, num_images=20, title="Validation samples")


## 5 · Train Classification Model

In [ ]:
model, history = train_classification_model(
    train_dir=clf_cfg.train_dir,
    val_dir=clf_cfg.val_dir,
    image_size=clf_cfg.image_size,
    batch_size=clf_cfg.batch_size,
    epochs=clf_cfg.epochs,
    learning_rate=clf_cfg.learning_rate,
    early_stopping_patience=clf_cfg.early_stopping_patience,
    dense_units=clf_cfg.dense_units,
    dropout_rate=clf_cfg.dropout_rate,
)


## 6 · Evaluate Classification Model

### 6.1 Training Curves

In [ ]:
def plot_training_history(history: tf.keras.callbacks.History) -> None:
    """Plot accuracy, recall, and loss curves from training history.

    Args:
        history: Object returned by ``model.fit()``.
    """
    metrics = [
        ("accuracy", "Accuracy"),
        ("recall", "Recall"),
        ("loss", "Loss"),
    ]
    fig, axes = plt.subplots(1, len(metrics), figsize=(5 * len(metrics), 4))
    for ax, (key, label) in zip(axes, metrics):
        ax.plot(history.history[key], label=f"Train {label}")
        ax.plot(history.history[f"val_{key}"], label=f"Val {label}")
        ax.set_xlabel("Epoch")
        ax.set_ylabel(label)
        ax.set_title(label)
        ax.legend()
    plt.tight_layout()
    plt.show()


plot_training_history(history)


### 6.2 Confusion Matrix & Classification Report

In [ ]:
def evaluate_with_threshold(
    model: tf.keras.Model,
    generator: ImageDataGenerator,
    threshold: float = 0.3,
) -> tuple[np.ndarray, np.ndarray]:
    """Collect all predictions and compute classification metrics.

    The model output is ``P(other)``.  A sample is predicted as *Bear* when
    ``1 - model_output >= threshold``, i.e. ``model_output < 1 - threshold``.

    Args:
        model: Trained Keras classification model.
        generator: Validation ``DirectoryIterator`` (``shuffle=False``).
        threshold: Minimum P(bear) to predict "Bear".

    Returns:
        Tuple of ``(predicted_class_indices, true_class_indices)``.
    """
    generator.reset()
    all_preds: list[float] = []
    all_true: list[float] = []

    for _ in range(len(generator)):
        images, true_labels = next(generator)
        preds = model.predict(images, verbose=0)
        all_preds.extend(preds.flatten().tolist())
        all_true.extend(true_labels.flatten().tolist())

    all_preds_arr = np.array(all_preds)
    all_true_arr = np.array(all_true, dtype=int)

    # predicted class: 0 (bear) when P(bear)=1-output >= threshold
    predicted = (all_preds_arr >= (1.0 - threshold)).astype(int)

    cm = confusion_matrix(all_true_arr, predicted)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Bear", "Other"])
    disp.plot(cmap=plt.cm.Blues)
    plt.title(f"Confusion Matrix  (threshold = {threshold})")
    plt.show()

    report = classification_report(all_true_arr, predicted, target_names=["Bear", "Other"])
    print(f"\nClassification Report (threshold = {threshold}):\n{report}")

    return predicted, all_true_arr


predicted_labels, true_labels = evaluate_with_threshold(
    model, val_generator, threshold=clf_cfg.threshold
)


### 6.3 Sample Predictions

In [ ]:
def plot_random_predictions(
    model: tf.keras.Model,
    generator: ImageDataGenerator,
    threshold: float = 0.3,
    num_images: int = 20,
) -> None:
    """Display a random grid of validation images with true and predicted labels.

    Args:
        model: Trained Keras model.
        generator: Validation ``DirectoryIterator`` (``shuffle=False``).
        threshold: Minimum P(bear) to predict "Bear".
        num_images: Number of images to display.
    """
    generator.reset()
    class_labels = {v: k for k, v in generator.class_indices.items()}

    all_images: list[np.ndarray] = []
    all_true: list[float] = []
    for _ in range(len(generator)):
        imgs, lbls = next(generator)
        all_images.append(imgs)
        all_true.append(lbls)

    all_images_arr = np.concatenate(all_images, axis=0)
    all_true_arr = np.concatenate(all_true, axis=0)

    raw_preds = model.predict(all_images_arr, verbose=0).flatten()
    # predicted class index: 0 (bear) when P(bear) >= threshold
    predicted_classes = (raw_preds >= (1.0 - threshold)).astype(int)

    num_images = min(num_images, len(all_images_arr))
    indices = np.random.choice(len(all_images_arr), num_images, replace=False)

    cols = 5
    rows = (num_images + cols - 1) // cols
    plt.figure(figsize=(15, rows * 3))
    for plot_idx, img_idx in enumerate(indices):
        true_name = class_labels[int(all_true_arr[img_idx])]
        pred_name = class_labels[int(predicted_classes[img_idx])]
        color = "green" if true_name == pred_name else "red"
        plt.subplot(rows, cols, plot_idx + 1)
        plt.imshow(all_images_arr[img_idx])
        plt.title(f"True: {true_name}\nPred: {pred_name}", fontsize=8, color=color)
        plt.axis("off")

    plt.tight_layout()
    plt.show()


plot_random_predictions(model, val_generator, threshold=clf_cfg.threshold, num_images=20)


## 7 · Run Inference on New Images

*(Colab only — uses `google.colab.files.upload()`)*

In [ ]:
def upload_and_predict(
    model: tf.keras.Model,
    threshold: float = 0.3,
    image_size: tuple[int, int] = (224, 224),
) -> None:
    """Interactive inference: upload images (or a ZIP), classify, and save results.

    Results are written to ``predictions/<timestamp>/predicted_bears/`` and
    ``predictions/<timestamp>/predicted_others/``.

    Args:
        model: Trained Keras classification model.
        threshold: Minimum P(bear) to label an image as "Bear".
        image_size: *(W, H)* expected by the model.
    """
    if not IN_COLAB:
        print("This cell requires Google Colab.")
        return

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    bear_dir = Path("predictions") / timestamp / "predicted_bears"
    other_dir = Path("predictions") / timestamp / "predicted_others"
    bear_dir.mkdir(parents=True, exist_ok=True)
    other_dir.mkdir(parents=True, exist_ok=True)

    uploaded = colab_files.upload()
    if not uploaded:
        print("No files uploaded.")
        return

    image_paths: list[Path] = []
    for filename in uploaded:
        fpath = Path(filename)
        if filename.endswith(".zip"):
            extracted = extract_zip(fpath, Path("uploaded_images"))
            image_paths.extend(collect_images_from_dir(extracted))
        elif fpath.suffix.lower() in {".jpg", ".jpeg", ".png"}:
            image_paths.append(fpath)
        else:
            print(f"Skipping unsupported file: {filename}")

    bear_count = other_count = 0
    for img_path in image_paths:
        batch = batch_from_path(img_path, target_size=image_size)
        if batch is None:
            print(f"[WARN] Could not load: {img_path.name}")
            continue

        label, bear_prob = predict_image(model, batch, threshold=threshold)
        dest_dir = bear_dir if label == "Bear" else other_dir
        shutil.copy(img_path, dest_dir / img_path.name)

        if label == "Bear":
            bear_count += 1
        else:
            other_count += 1

        print(f"{img_path.name:40s}  →  {label}  (P(bear)={bear_prob:.3f})")

    total = bear_count + other_count
    print(f"\nDone — {total} images | Bear: {bear_count} | Other: {other_count}")
    print(f"Results saved in predictions/{timestamp}/")


upload_and_predict(model, threshold=clf_cfg.threshold, image_size=clf_cfg.image_size)


## 8 · YOLOv8 Bear Detection Model

### 8.1 Dataset Setup

In [ ]:
import os

# ── Option A: Use the local Roboflow dataset included in the repo ─────────────
LOCAL_DATASET_DIR = "Bear detection.v3i.yolov8-obb"
data_yaml_path = setup_local_dataset(LOCAL_DATASET_DIR)
print("data.yaml:", data_yaml_path)

# ── Option B: Download a custom dataset from Roboflow ─────────────────────────
# api_key = os.environ.get("ROBOFLOW_API_KEY")   # set via Colab Secrets or .env
# if api_key is None:
#     raise EnvironmentError("ROBOFLOW_API_KEY not set.")
# data_yaml_path = setup_roboflow_dataset(
#     api_key=api_key,
#     workspace="your-workspace",
#     project="your-project",
#     version=1,
# )


### 8.2 Train or Load Weights

In [ ]:
USE_PRETRAINED = False  # Set to True and provide WEIGHTS_PATH to skip training

WEIGHTS_PATH = "best.pt"  # Path to your .pt weights file

if USE_PRETRAINED:
    from ultralytics import YOLO
    yolo_model = YOLO(WEIGHTS_PATH)
    print("Loaded pre-trained weights from", WEIGHTS_PATH)
else:
    yolo_model = train_detection_model(
        data_yaml=data_yaml_path,
        model_name=det_cfg.model_name,
        epochs=det_cfg.epochs,
        image_size=det_cfg.image_size,
        batch_size=det_cfg.batch_size,
        optimizer=det_cfg.optimizer,
        learning_rate=det_cfg.learning_rate,
        weight_decay=det_cfg.weight_decay,
        augment=det_cfg.augment,
        half_precision=det_cfg.half_precision,
    )
    print("Training complete.")


### 8.3 Process Video

In [ ]:
def run_video_inference(yolo_model) -> None:
    """Upload a video (Colab), run YOLOv8 detection, and save the annotated output."""
    if not IN_COLAB:
        print("This cell requires Google Colab.")
        return

    video_dir = Path("video_files")
    output_dir = Path("processed_videos")
    video_dir.mkdir(exist_ok=True)
    output_dir.mkdir(exist_ok=True)

    print("Upload your video file(s) (.mp4):")
    uploaded = colab_files.upload()

    for filename, data in uploaded.items():
        video_path = video_dir / filename
        video_path.write_bytes(data)

        output_path = output_dir / filename.replace(".mp4", "_detected.mp4")
        print(f"Processing {filename} …")
        n_frames = process_video(video_path, yolo_model, output_path=output_path)
        print(f"Done — {n_frames} frames  →  {output_path}")


run_video_inference(yolo_model)


## 9 · Save / Export Trained Weights

*(Colab only)*

In [ ]:
if IN_COLAB:
    # Classification model (Keras .keras format)
    model.save("bear_classifier.keras")
    colab_files.download("bear_classifier.keras")

    # YOLOv8 weights (best.pt lives inside runs/detect/train/weights/)
    import glob as _glob
    best_pts = _glob.glob("runs/detect/train*/weights/best.pt")
    if best_pts:
        shutil.make_archive("yolo_weights", "zip", os.path.dirname(best_pts[0]))
        colab_files.download("yolo_weights.zip")
    else:
        print("No YOLOv8 weights found. Make sure training completed.")
else:
    print("Save manually: model.save('bear_classifier.keras')")
